In [1]:
def prox_norm_2( x, tau ) :
    n = x.size
    y = np.zeros( n )
    nx = linalg.norm( x )
    if nx >= tau or nx != 0 :
        y = ( 1 - tau / nx ) * x

    return y

def prox_norm_1( x, tau ) :
    n = x.size
    y = np.zeros( n )
    for i in range( 0, n ):
        if x[i] >= tau :
            y[i] = x[i] - tau
        elif x[i] <= -tau:
            y[i] = x[i] + tau
            
    return y

In [ ]:
def line_search_wolfe( x, a, p, f, g, c1, c2, m ) :
  alpha_min = 0
  alpha = a
  alpha_max = np.inf

  fx = f( x )
  gx = g( x )
  hx = gx.dot( p )

  z = x + alpha * p
  fz = f( z )
  gz = g( z )
  hz = gz.dot( p ) 

  W1 = fx + alpha * c1 * hx
  W2 = c2 * hx
  i = 0
  while ( ( fz > W1 or hz < W2 ) and i < m ) :
    if fz > W1:
      alpha_max = alpha
      alpha = 0.5 * ( alpha_min + alpha_max )

    elif hz < W2 :
      if alpha_max >= np.inf :
        alpha = 2 * alpha
      else :
        alpha_min = alpha
        alpha = 0.5 * ( alpha_min + alpha_max )

    z = x + alpha * p
    fz = f( z )
    gz = g( z )
    hz = gz.dot( p )
    i = i + 1

  chk = fz > W1 or hz < W2

  return ( alpha, i, chk )